In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
import datetime

import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt

import http.client
import urllib.parse
import os
import requests
import json


In [ ]:
access_token = "pk.eyJ1Ijoiam95eHUiLCJhIjoiY2w4NXlkMmFzMHBnZTQxcXRtaXdqb2FmOCJ9.fmZUE9d1q1VmedH00fCoLA"
data_path = "/u1/joyxu/pet_projects/pag-foodies/data/foodie-data.csv"
reviews_path = "/u1/joyxu/pet_projects/pag-foodies/data/reviews.csv"

In [ ]:
geocoding_token = "3d111c9b7312aae9282e1af8152009ba"
os.environ["http_proxy"] = "http://httpproxy.blackrock.com:8080/"
os.environ["https_proxy"] = "https://httpsproxy.blackrock.com:8080/"

In [ ]:
#data = pd.read_excel(data_path,engine='openpyxl')
if 'data' not in st.session_state:
    st.session_state.data = pd.read_csv(data_path)
if 'reviews' not in st.session_state:
    st.session_state.reviews = pd.read_csv(reviews_path)

In [ ]:
def format_type(food_type: str):
    types_dict = {
        "lunch" : "Lunch",
        "coffee" : "Coffee",
        "pastries" :  "Pastries",
        "drinks" : "Drinks"
    }
    
    return types_dict[food_type]

def encode_types(food_types: list):
    lst = [0] * (len(st.session_state.data.columns) - 4) # minus lat, long, coords and name
    type_to_idx = {
        "lunch" : 0,
        "coffee" : 1,
        "pastries" :  2,
        "drinks" : 3
    }
    
    for food_type in food_types:
        lst[type_to_idx[food_type]] = 1
        
    return lst

In [ ]:
st.title("Joy's Foodie Guide to BlackRock 🍱")
st.write("For the days when you don't know where to go!")

In [ ]:
refresh = st.button('Refresh data')
if refresh:
    st.session_state.data = pd.read_csv(data_path)
    st.session_state.reviews = pd.read_csv(reviews_path)

In [ ]:
# daily rec

In [ ]:
# surprise me
surprise = st.button('Surprise me!')
if surprise:
        n = np.random.randint(len(st.session_state.data))
        st.write("Go to... " + st.session_state.data.iloc[n]["name"])

In [ ]:
# gimme a place
st.header("Give me a place for...")

occasions = st.session_state.data.drop(['name','coordinates','longitude','latitude'], axis=1).columns
occasion = st.selectbox("Choose an occasion", list(occasions), format_func=format_type)
options = st.session_state.data[st.session_state.data[occasion]==1]
st.write("Here are some options for {}".format(occasion))
show_options = options["name"].tolist()
for i in show_options:
    st.markdown("- " + i)

In [ ]:
# show reviews
st.header("Reviews ⭐")

place = st.selectbox("Show reviews for...", st.session_state.data['name'])
selected_reviews = st.session_state.reviews.loc[st.session_state.reviews['name']==place]
if selected_reviews.empty:
    st.write("No reviews to show! Why don't you add one below? :)")
else:
    st.write(selected_reviews)

In [ ]:
# map
st.header("Map 📌")

px.set_mapbox_access_token(access_token)
fig = px.scatter_mapbox(st.session_state.data, lat="latitude", lon="longitude",color="name",
                  color_continuous_scale=px.colors.cyclical.IceFire, size_max=20,zoom=12)
st.write(fig)

In [ ]:
# add your recs (form)

with st.form("recs", clear_on_submit = True):
    st.header("Give a recommendation 💯")
    name = st.text_input("Name")
    search = st.text_input("Enter your address")
    st.caption("Please write the address in the following format: 100 Middlesex St, London E1 7EZ")
    # st.caption("Find the coordinates of the address [here](https://www.latlong.net/).")
    # lat = st.number_input("Latitude", step=1e-7, format="%.6f")
    # long = st.number_input("Longitude", step=1e-7, format="%.6f")
    occ = st.multiselect("Good for...", ['lunch','coffee','pastries','drinks'], format_func=format_type)
    submit = st.form_submit_button("Submit")
    
    if submit:
        conn = http.client.HTTPConnection('api.positionstack.com')

        params = urllib.parse.urlencode({
            'access_key': geocoding_token,
            'query': search,
            'country': 'GB',
            'region': 'London',
            'limit': 1,
            })

        r = requests.get('http://api.positionstack.com/v1/forward?{}'.format(params), 
                         proxies={"http": "http://httpproxy.blackrock.com:8080/"})

        st.session_state.res = json.loads(r.text)

if 'res' in st.session_state:
    res_lat = st.session_state.res['data'][0]['latitude']
    res_long = st.session_state.res['data'][0]['longitude']
    res_neighbourhood = st.session_state.res['data'][0]['neighbourhood']
    res_adr = st.session_state.res['data'][0]['label']
    coords = (res_lat,res_long)

    st.write(f"Neighbourhood: {res_neighbourhood}")
    st.write(f"Address: {res_adr}")
    st.write(f"Coordinates: {coords}")

    correct = st.checkbox("This is correct", )
    incorrect = st.checkbox("Manually edit coordinates", )
    if correct:
        one_hot_lst = encode_types(occ)
        st.session_state.data.loc[len(st.session_state.data.index)] = [name,res_lat,res_long,coords] +  one_hot_lst
        st.session_state.data = st.session_state.data.drop_duplicates(subset="coordinates")
        st.session_state.data.to_csv(data_path, index=False)
        st.write("Thank you for the recommendation!")
        
    if incorrect:
        with st.form("change", clear_on_submit = False):
            st.caption("Find the coordinates of the address [here](https://www.latlong.net/).")

            lat = st.number_input("Latitude", step=1e-7, format="%.6f", value=res_lat)
            long = st.number_input("Longitude", step=1e-7, format="%.6f", value=res_long)
            coords = (lat,long)

            edit = st.form_submit_button("Submit edits")
        if edit:
            one_hot_lst = encode_types(occ)
            st.session_state.data.loc[len(st.session_state.data.index)] = [name,lat,long,coords] +  one_hot_lst
            st.session_state.data = st.session_state.data.drop_duplicates(subset="coordinates")
            st.session_state.data.to_csv(data_path, index=False)
            st.write("Thank you for the recommendation!")

In [ ]:
# add a review
with st.form("review", clear_on_submit = True):
    st.header("Review a place 📎")
    place = st.selectbox("Choose a place to review", st.session_state.data['name'])
    review = st.text_area("Your review: ")
    submit_review = st.form_submit_button("Submit")
    
if submit_review:
    st.session_state.reviews.loc[len(st.session_state.reviews.index)] = [place, review] 
        
    st.session_state.reviews.to_csv(reviews_path, index=False)
    st.write("Thank you for the review!")

In [ ]:
# delete a place
with st.form("delete", clear_on_submit = True):
    st.header("Delete a place 📎")
    place = st.selectbox("Choose a place to delete", st.session_state.data['name'])
    submit_delete = st.form_submit_button("Submit")